SAM-HQ
====

**Segment Anything in High Quality**
(aka HQ-SAM)

 * Paper: https://arxiv.org/abs/2306.01567

![SAM-HQ Model Architrcture](../assets/samhq_arch.jpg)

 * Installation

```bash
conda create -n samhq python=3.12 -y
pip install torch torchvision
pip install segment-anything-hq
pip install timm huggingface-hub matplotlib
```

 * Download model

```bash
from huggingface_hub import hf_hub_download

# Repository and filename
repo_id = "lkeab/hq-sam"
filename = "sam_hq_vit_b.pth"

# Download the file
model_path = hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir="checkpoints",
    local_dir_use_symlinks=False
)

print(f"Model downloaded to: {model_path}")
```

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
from segment_anything_hq import (
    sam_model_registry,
    SamPredictor
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "vit_b"
checkpoint_path = "checkpoints/sam_hq_vit_b.pth"

sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
sam.to(device=device)

predictor = SamPredictor(sam)
predictor.model.eval();

/home/pyml/anaconda3/envs/samhq/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pyml/anaconda3/envs/samhq/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/pyml/anaconda3/envs/samhq/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/pyml/anaconda3/envs/samhq/lib/python3.12/site-packages/segment_anything_hq/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_5m_

<All keys matched successfully>
